# imports

In [1]:
# all imports
# for importing dataset
from keras.datasets import cifar10

import sys
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, SpatialDropout2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD, Adam
from keras.layers import Dropout

# from keras.constraints import maxnorm

from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation
from keras.layers import LeakyReLU

# # Load data

In [2]:
# load & plot data

# Folloed https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
# to create a testing environtment for varoious deep learning algorithms.

def load_dataset():
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

def prep_pixels(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    train_norm = train_norm/255.0
    test_norm = test_norm/255.0
    return train_norm, test_norm

#  for visualization of results followed this tutorial 
# https://www.kaggle.com/danbrice/keras-plot-history-full-report-and-grid-search
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# AlexNet

In [ ]:
def alexNet():
    model = Sequential()
    
    model.add(Conv2D(filters=96, kernel_size=(3,3), strides=4, input_shape=(32,32,3), activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Conv2D(256, (5,5), padding='same', activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Conv2D(384, (3,3), padding='same', activation='relu'))
    
    model.add(Conv2D(384, (3,3), padding='same', activation='relu'))
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(4096, activation='relu'))
#     model.add(Dense(4096))
#     model.add(LeakyReLU(alpha=0.05))
    
#     model.add(Dropout(0.3))  #0.2 0.5
    
    model.add(Dense(4096, activation='relu'))   
#     model.add(Dense(4096))
#     model.add(LeakyReLU(alpha=0.05))        #act. func. leaky relu

#     model.add(Dropout(0.3))  #0.2 0.5
    
    model.add(Dense(10, activation='softmax'))  
    
    
#     opt = SGD(lr=0.001, momentum=0.9)       # lr =0.0003, default
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# test the network
trainX, trainY, testX, testY = load_dataset()

# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)

# define model
model = alexNet()

# fit model  batch_size=64
history = model.fit(trainX, trainY, epochs=20, batch_size=256, validation_data=(testX, testY), verbose=1)

# evaluate model
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))

# plot
plot_history(history)

# entry point, run the test harness
# run_test_harness()

# VGG16

## following vgg without batch normalization

In [7]:
def vgg():
    model = Sequential()
    
    model.add(Conv2D(64,(3,3),activation='relu',input_shape=(32,32,3),padding='same'))
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(4096,activation='relu'))
    model.add(Dense(4096,activation='relu'))
    
    model.add(Dense(10))
    
    model.add(Activation('softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.0003), metrics=['accuracy'])
#     sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=['accuracy'])
    model.summary()
    return model

model = vgg()

## below VGG with batch normalization

In [ ]:
def vgg():
    model = Sequential()
    
    model.add(Conv2D(64,(3,3),activation='relu',input_shape=(32,32,3),padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(4096,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(4096,activation='relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(10))
    
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.0003), metrics=['accuracy'])
    model.summary()
    return model

## following vgg with batch normalization and spatialdropout

In [ ]:
def vgg():
    model = Sequential()
    
    model.add(Conv2D(64,(3,3),activation='relu',input_shape=(32,32,3),padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(SpatialDropout2D(0.5))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(SpatialDropout2D(0.5))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(SpatialDropout2D(0.5))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(SpatialDropout2D(0.5))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    
    model.add(SpatialDropout2D(0.5))
    
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(4096,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(4096,activation='relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(10))
    
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.0003), metrics=['accuracy'])
    model.summary()
    return model

In [9]:
# test the network
trainX, trainY, testX, testY = load_dataset()

# prepare pixel data
# trainX, testX = prep_pixels(trainX, testX)

# define model
model = vgg()

# fit model
history = model.fit(trainX, trainY, epochs=10, batch_size=64, validation_data=(testX, testY), verbose=1)

# evaluate model
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))

# plot
plot_history(history)


Epoch 1/10
782/782 [==============================] - 21s 26ms/step - loss: 3.5632 - accuracy: 0.0974 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/10
782/782 [==============================] - 20s 26ms/step - loss: 2.3027 - accuracy: 0.1001 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/10
159/782 [=====>........................] - ETA: 15s - loss: 2.3027 - accuracy: 0.0923

KeyboardInterrupt: 